$$\textrm{Joaquin Peñuela Parra, Cristian Fernando Rodriguez Cruz}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

In [1]:
import os
import subprocess
from itertools import product
## IMPORTANT: Make sure that "Uniandes_Framework" is in .gitignore
framework_path = "Uniandes_Framework"

if os.path.exists(framework_path):
    # Pull updates if the framework is already cloned
    try:
        subprocess.run(["git", "-C", framework_path, "pull"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while pulling updates from the framework: {e}")
else:
    # Clone the framework if it is not already cloned
    try:
        subprocess.run(["git", "clone", "git@github.com:Phenomenology-group-uniandes/Uniandes_Framework.git"])
    except subprocess.CalledProcessError as e:
        raise Exception(f"Error occurred while cloning the framework: {e}")

hint: Pulling without specifying how to reconcile divergent branches is
hint: discouraged. You can squelch this message by running one of the following
hint: commands sometime before your next pull:
hint: 
hint:   git config pull.rebase false  # merge (the default strategy)
hint:   git config pull.rebase true   # rebase
hint:   git config pull.ff only       # fast-forward only
hint: 
hint: You can replace "git config" with "git config --global" to set a default
hint: preference for all repositories. You can also pass --rebase, --no-rebase,
hint: or --ff-only on the command line to override the configured default per
hint: invocation.


Already up to date.


In [2]:
import os

from Uniandes_Framework.delphes_reader import DelphesLoader 
from Uniandes_Framework.delphes_reader import classifier 
from Uniandes_Framework.delphes_reader import root_analysis 
from Uniandes_Framework.delphes_reader import Quiet 

from ROOT import TChain
from ROOT import TLorentzVector

import pandas as pd
from multiprocessing import Pool

Welcome to JupyROOT 6.22/06


In [3]:
case = "woRHC"
sim_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))), "SIMULATIONS", f"lq_zp_{case}")
signals = ["ta_ta","zp_tau_tau","lq_lq","lq_ta"]
xs_path = { signal: os.path.join( os.path.dirname(os.getcwd()), "01_signal_production", f"xs_signals_{case}", signal, "XS_Matrix.csv") for signal in signals}
                       
M_U= [1000,1500]
G_U = [1.5]

csv_sim_path = pd.read_csv(os.path.join("Uniandes_Framework", "SimulationsPaths.csv"))
df = pd.concat(
    [csv_sim_path]
    +
    [ pd.DataFrame.from_dict(
    {
        "name" : signal + f"_m{m}_g{g}".replace('.', '_'), 
        "path": os.path.join(sim_path, signal,f'M{m}_gU{g:.4f}'.replace('.', '_')),  
        'xs(pb)': pd.read_csv(xs_path[signal], index_col = 0).loc[g,str(m)]
    }, 
    orient = "index").T for signal, m , g in product(signals,M_U,G_U) ]
)
# save the new csv
csv_sim_path = os.path.join(os.getcwd(), "SimulationsPaths.csv")
df.to_csv(csv_sim_path, index = False)

In [4]:
def event_selection(signal):
    
    csv_folder_path = f'/disco4/pheno_csv_files/Leptoquarks_Searches/{signal}/'
    
    with Quiet():
    
        CUTS={"l_jet":     {"pt_min_cut":30., "eta_min_cut":-5.,  "eta_max_cut":+5. },
              "b_jet":     {"pt_min_cut":30., "eta_min_cut":-2.4, "eta_max_cut":+2.4},
              "tau_jet":   {"pt_min_cut":50., "eta_min_cut":-2.3, "eta_max_cut":+2.3},
              "other_jet": {"pt_min_cut":0.,  "eta_min_cut":-5.,  "eta_max_cut":+5.},
              "electron":  {"pt_min_cut":35.,  "eta_min_cut":-2.4, "eta_max_cut":+2.4},
              "muon":      {"pt_min_cut":30.,  "eta_min_cut":-2.4, "eta_max_cut":+2.4}}

        ST = []
        
        Delphes = DelphesLoader(signal,csv_sim_path)
        Paths = Delphes.Forest
        xs = Delphes.xs   
        
        #We want a Cutflows file for each cathegory:        
        Cutflows = {'hadronic_non-resonant': {'XS': xs},
                    'hadronic_sLQ': {'XS': xs},
                    'hadronic_dLQ': {'XS': xs},
                    'semileptonic_non-resonant': {'XS': xs},
                    'semileptonic_sLQ': {'XS': xs},
                    'semileptonic_dLQ': {'XS': xs}}
        
        kinematic_variables = {}
        for key in Cutflows.keys(): kinematic_variables[key] = []
        i=0
        for path in Paths:
            i+=1
            tree = TChain("Delphes;1") 
            tree.Add(path) 
            print("Working on: ", path)
            for event in tree:
                
                for key in Cutflows.keys(): Cutflows[key]['All'] = Cutflows[key].get('All',0) + 1
                    
                leptons = classifier.get_good_leptons(event, kin_cuts= CUTS) 
                
                if(len(leptons) > 1): continue
                for key in Cutflows.keys(): Cutflows[key]['Maximum 1 light lepton'] = Cutflows[key].get('Maximum 1 light lepton',0) + 1
                           
                jets = classifier.get_good_jets(event, kin_cuts= CUTS)
                
                #Adding tau_jets like leptons:
                leptons = leptons + jets['tau_jet'] 
                
                if(len(leptons) != 2): continue 
                for key in Cutflows.keys(): Cutflows[key]['Exactly 2 leptons'] = Cutflows[key].get('Exactly 2 leptons',0) + 1

                #At this point, we have two hadronic taus or one hadronic tau and one lepton. Now, we can clasificate:
                
                #clasification:
                if(len(jets['tau_jet']) == 1): 
                    label1 = 'semileptonic'
                else:
                    label1 = 'hadronic'  
                
                    
                cut = f'{label1} selection'    
                for key in Cutflows.keys(): 
                    if label1 in key: Cutflows[key][cut] = Cutflows[key].get(cut, 0) + 1
                
                if(len(jets['b_jet']) == 0):
                    label2 = 'non-resonant'
                elif(len(jets['b_jet']) == 1):
                    label2 = 'sLQ'
                elif(len(jets['b_jet']) == 2):
                    label2 = 'dLQ'
                else: continue
                
                label = f'{label1}_{label2}'  
                
                cut = f'{label2} selection'    
                Cutflows[label][cut] = Cutflows[label].get(cut, 0) + 1                
                
                if(leptons[0].DeltaR(leptons[1]) < 0.3): continue #DeltaR > 0.3
                cut = 'DeltaR > 0.3'
                Cutflows[label][cut] = Cutflows[label].get(cut,0) + 1
                
                MET = classifier.get_met(event)
                MET.SetName('MET')
                particles = leptons + jets['b_jet'] + [MET]
                
                ST = 0
                for particle in particles: ST += particle.pt
                
                HT = 0
                for particle in jets['l_jet']: HT += particle.pt
                
                TLV = TLorentzVector(0,0,0,0)
                for particle in particles: TLV += particle.GetTLV()
                MT = TLV.Mt()
                
                Mass_invariant = (leptons[0].GetTLV() + leptons[1].GetTLV()).M()
                
                row = root_analysis.get_kinematics_row(particles)
                
                keylist = list(row.keys())
                for key in keylist: 
                    if ("Mass" in key): row.pop(key,None)
                
                row["sT(GeV)"] = ST
                row["hT(GeV)"] = HT
                row["mT(GeV)"] = MT
                row["light_jets_multiplicity"] = len(jets['l_jet'])
                name = leptons[0].Name + leptons[1].Name
                row[f"Mass_{{{name}}}(GeV)"] = Mass_invariant
                row[f"Q_{{{leptons[0].Name}}}Q_{{{leptons[1].Name}}}"] = leptons[0].Charge*leptons[1].Charge
                
                
                kinematic_variables[label].append(row)
            break
        !mkdir -p {csv_folder_path}                           
        for key in Cutflows.keys(): root_analysis.generate_csv(kinematic_variables[key], os.path.join(csv_folder_path, f'{signal}_{key}.csv'))
                
    return {signal: Cutflows}

In [5]:
signals = ["ta_ta_m1000_g1_5","zp_tau_tau_m1000_g1_5"]

In [ ]:
with Pool(4) as p:
    mapping = list(map(event_selection, signals))
#list(map(event_selection, signals))


ta_ta_m1000_g1_5 imported with 12 trees!
/home/Cristian/SIMULATIONS/lq_zp_woRHC/ta_ta/M1000_gU1_5000
Working on:  /home/Cristian/SIMULATIONS/lq_zp_woRHC/ta_ta/M1000_gU1_5000/Events/run_01/tag_1_delphes_events.root
hadronic_non-resonant non-resonant
tau_jet_{0}
tau_jet_{1}
MET
semileptonic_sLQ sLQ
lep_{1}
tau_jet_{0}
b_jet_{2}
MET
hadronic_sLQ sLQ
tau_jet_{0}
tau_jet_{1}
b_jet_{3}
MET
hadronic_non-resonant non-resonant
tau_jet_{0}
tau_jet_{2}
MET
semileptonic_non-resonant non-resonant
lep_{1}
tau_jet_{0}
MET
semileptonic_non-resonant non-resonant
lep_{1}
tau_jet_{1}
MET
hadronic_non-resonant non-resonant
tau_jet_{0}
tau_jet_{1}
MET
hadronic_non-resonant non-resonant
tau_jet_{0}
tau_jet_{1}
MET
semileptonic_non-resonant non-resonant
lep_{1}
tau_jet_{0}
MET
hadronic_non-resonant non-resonant
tau_jet_{0}
tau_jet_{1}
MET
semileptonic_non-resonant non-resonant
lep_{1}
tau_jet_{0}
MET
hadronic_sLQ sLQ
tau_jet_{0}
tau_jet_{1}
b_jet_{2}
MET
semileptonic_non-resonant non-resonant
lep_{1}
tau_jet

In [ ]:
labels = ['hadronic_non-resonant','hadronic_sLQ','hadronic_dLQ','semileptonic_non-resonant','semileptonic_sLQ','semileptonic_dLQ']

Dict_Cutflows = {}

for label in labels: Dict_Cutflows[label] = {}

for Cutflows_Signal_Directory in mapping:
    
    signal = list(Cutflows_Signal_Directory.keys())[0]
    
    for label in labels: Dict_Cutflows[label][signal] = Cutflows_Signal_Directory[signal][label]

In [ ]:
csv_folder_path = f'/disco4/pheno_csv_files/Leptoquarks_Searches/'

Dict_DataFrames_Cutflows = {}

for key in Dict_Cutflows: 
    Dict_DataFrames_Cutflows[key] = pd.DataFrame.from_dict(Dict_Cutflows[key])
    carpeta = os.path.join(csv_folder_path, 'cutflows_lq_zp')
    os.makedirs(carpeta, exist_ok=True)
    Dict_DataFrames_Cutflows[key].to_csv(os.path.join(carpeta, f'{key}.csv'))